# キャップストーン: SageMaker Studio と SageMaker Python SDK を使用して、エンドツーエンドの表形式データ機械学習プロジェクトを構築する

## 環境の設定

ラボの開始に必要となる、基本的な設定コードは以下に用意されています。パッケージのインストールと変数の作成のために、まずこれらのセルを読み込んで実行します。

In [ ]:
%%capture
%pip install -U shap
%pip install -U smdebug
%pip install imbalanced-learn
%pip install pytest-cov
%pip install pytest-filter-subpackage
%pip install sagemaker
%pip install -U seaborn

In [ ]:
#必要なライブラリ

import boto3
import datetime as datetime
import io
import IPython
import json
import math
import matplotlib.pyplot as plt  # visualization
import numpy as np
import pandas as pd
import pathlib
import re
import sagemaker
import seaborn as sns  # visualization
import statistics
import string
import sys
import time
import zipfile

from imblearn.over_sampling import SMOTE

from sagemaker import clarify
from sagemaker import get_execution_role
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.dataset_definition.inputs import AthenaDatasetDefinition, DatasetDefinition, RedshiftDatasetDefinition
from sagemaker.debugger import CollectionConfig, DebuggerHookConfig, FrameworkProfile, ProfilerConfig, ProfilerRule, Rule, rule_configs
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
from sagemaker.feature_store.feature_definition import FeatureDefinition
from sagemaker.feature_store.feature_definition import FeatureTypeEnum
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.inputs import CreateModelInput
from sagemaker.inputs import TrainingInput
from sagemaker.inputs import TransformInput
from sagemaker.model import Model
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.network import NetworkConfig
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput, ScriptProcessor
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.s3 import S3Uploader
from sagemaker.session import Session
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.transformer import Transformer
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.workflow.condition_step import ConditionStep, JsonGet
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.steps import TuningStep
from sagemaker.xgboost.estimator import XGBoost
from sklearn.model_selection import train_test_split

In [ ]:
# 基本的な変数とコードの設定

%matplotlib inline
base_job_name = "capstone-smdebugger-job"
bucket = sagemaker.Session().default_bucket()
bucket_path = "s3://{}".format(bucket)
prefix = "sagemaker/capstone"
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
s3_client = boto3.client("s3")
sagemaker_session = sagemaker.Session()
save_interval = 5
sns.set(color_codes=True)

## データセットの説明

Amazon Simple Storage Service (Amazon S3) バケットに、以下の 5 つのテーブルが保存されています。
- **claims.csv**: 保険金請求の raw データを含むテーブル
- **customers.csv**: 顧客の raw データを含むテーブル
- **claims_preprocessed.csv**: 保険金請求の処理済みデータを含むテーブル
- **customers_preprocessed.csv**: 顧客の処理済みデータを含むテーブル
- **claims_customer.csv**: 処理済みの保険金請求データと顧客データを **policy_id** で結合したテーブル

このラボでは、まず **claims.csv** と **customers.csv** のテーブルから見ていきます。**課題 1** で、Amazon SageMaker Data Wrangler を使用し、これらのテーブルを処理します。作業がうまくいかない場合や、処理済みのデータセットを参照する場合は、処理済みのテーブルの内容を確認できます。

このデータセットでは、claims.csv テーブルの **fraud** 列がターゲットです。

claims.csv テーブルには、以下のフィールドがあります。

- **policy_id**: 保険契約の固有 ID
- **driver_relationship**: ドライバーとの関係性に関するリスト (配偶者、本人、子ども、その他、該当なし)
- **incident_type**: 報告された事故のタイプ (侵入、衝突、盗難)
- **collision_type**: 衝突した場所 (前面、背面、側面、該当なし)
- **incident_severity**: 事故の重大度 (軽微、重大、全損)
- **authorities_contacted**: 最初に通報した公的機関 (なし、警察、救急車、消防)
- **num_vehicles_involved**: 事故に巻き込まれた車両の数 (1～6 台の範囲)
- **num_injuries**: 事故による負傷者の数 (1～4 人の範囲)
- **num_witnesses**: 事故の目撃者の数 (1～5 人の範囲)
- **police_report_available**: 警察の報告書があるかどうか (はい、または いいえ)
- **injury_claim**: 傷害に対する保険金請求額 (米ドル) (300～576,300 USD)
- **vehicle_claim**: 車両損害に対する保険金請求額 (米ドル) (1,000～51,051 USD)
- **total_claim_amount**: 傷害と車両損害に対する保険金請求額の合計 (2,100～588,868 USD)
- **incident_month**: 事故が発生した月 (1～12 の範囲)
- **incident_day**: 事故が発生した日 (1～31 の範囲)
- **incident_dow**: 事故が発生した曜日 (日曜日から土曜日を表す 0～6 の範囲)
- **incident_hour**: 事故が発生した時刻 (0～23 の範囲)
- **fraud**: 保険契約が不正であったかどうか (0 または 1)

customers.csv テーブルには、以下のフィールドがあります。

- **policy_id**: 保険契約の固有 I
- **customer_age**: 当該顧客の年齢 (18～70 歳の範囲)
- **months_as_customer**: この顧客が保険料を支払った月数 (1～495 の範囲)
- **num_claims_past_year**: 当該顧客が過去 1 年間に行った保険金請求の件数
- **num_insurers_past_5_years**: 当該顧客が過去 5 年間に加入していた保険会社数
- **policy_state**: 当該顧客が住んでいる州 (AZ、CA、ID、NV、OR、WA)
- **policy_deductable**: 保険の免責金額 (米ドル) (750～1,100 USD の範囲)
- **policy_annual_premium**: 年間保険料 (米ドル) (2,200～3,000 USD の範囲)
- **policy_liability**: 対人賠償の上限額。単一の対人賠償およびすべての対人賠償額の上限額 (15/30、25/50、60/90、100/200)
- ***customer_zip**: 当該顧客の郵便番号 (83201～99362 の範囲)
- **customer_gender**: 当該顧客の性別 (男性、女性、その他、不明)
- **customer_education**: 当該顧客の教育レベル (高校未満、高校、準学士号、学士号、上級学位)
- **auto_year**: 車両の製造年 (2001～2020 の範囲)

これらのテーブルを、**policy_id** 列を使用した 内部結合で結合します。

## 課題ラボのナビゲーション

このラボには、課題のタスクやノートブックの最後にある付録に移動するためのリンクが用意されています。付録の項目を確認するには、解説のハイパーリンクをクリックします。現在作業中の課題に戻るには、付録に記載されている、対応するタスクのハイパーリンクをクリックします。

このラボは、以下のように構成されています。

- 課題 1: SageMaker Data Wrangler を使用してデータセットを分析して準備する
- 課題 2: SageMaker Feature Store で特徴量グループを作成する
- 課題 3: モデルをトレーニングする
- 課題 4: モデルのバイアスを評価する
- 課題 5: バッチ変換
- 課題 6: 自動化されたパイプラインを構築する
- 付録

## 課題 1: SageMaker Data Wrangler を使用してデータセットを分析して準備する

AnyCompany Consulting は、自動車保険詐欺のデータセットを分析し、新規の保険金請求について詐欺の可能性があるかどうかを予測するのに役立つモデルを構築したいという依頼を受けました。同社は、それぞれの保険金請求が不正かどうかをラベル付けした 5,000 件の顧客レコードを保有しています。このデータを使って、新しいレコードに対しバッチ推論を実行する前に、モデルのトレーニング、テスト、検証を行うことができます。

Amazon SageMaker Data Wrangler の分析機能を使用して、重要な列のデータ分布を可視化し、列間の相関をチェックし、データリーケージをチェックします。次に、簡単なベースラインモデルを構築します。そして、SageMaker Data Wrangler のデータ処理機能を使用して、よりパフォーマンスの高いモデルトレーニングに適するように列を変換します。

このタスクを完了するために、以下のサブタスクを実行します。

- データを表示する。
- Amazon SageMaker Studio で探索的データ分析を完了する。
- Amazon SageMaker Clarify のプロセッサジョブを使用してバイアスレポートを実行する。
- データを準備する。

この課題の所要時間は約 _100_ 分です。

### タスク 1.1: データを表示する

<a id="task1-1-continue"></a>

リポジトリに保存されている自動車保険の表形式データセットにアクセスし、データセットのサンプルを表示します。このリポジトリには、顧客データ (**customers.csv**) と保険金請求データ (**claims.csv**) の 2 つの未処理テーブルが含まれています。

**ヒント 1**: 未処理のテーブルは、**./data/** フォルダに保存されています。

**ヒント 2**: **claims.csv** と **customers.csv** テーブルは、未処理のテーブルです。

少し時間を取ってテーブルを確認してください。目立ったフィールドはありますか。 慎重な前処理が必要なフィールドはありますか。

データの表示方法については、「<a href="#task1-1" target="_self">**データを表示する**</a>」を参照してください。

自動車保険詐欺のテーブルにアクセスし、データセットのサンプルを確認したら、このタスクは完了です。

In [ ]:
# ここにタスク 1.1 のコードを追加する



### タスク 1.2: SageMaker Studio で探索的データ分析を完了する

<a id="task1-2-continue"></a>

データを表示し、データセット内の潜在的な問題を特定し、ターゲットに対して強い相関がある列があるかどうかを確認したら、探索的データ分析は完了です。データは SageMaker Data Wrangler とノートブックで調査できます。

具体的には、以下の項目について時間を取って確認してください。
- **列のヒストグラム**: 列を視覚的に表示し、データセットにどのような値が含まれているかを確認します。
- **クイックモデル**: データセットを見て、予想されるモデルの結果を考えます。
- **特徴量の相関**: ターゲットと強い相関がある列があるかどうかを確認します。
- **データリーケージ**: ターゲット値に依存するデータがあるかどうかを確認します。

SageMaker Data Wrangler を開くと、SageMaker Studio の新しいタブが開きます。以下のオプション２つのうち、どちらか1 つを実行して手順を進めます。
- **オプション 1:** タブを並べて表示します。メインの SageMaker Studio ウィンドウから分割画面ビューを作成するには、[**capstone_ja_jp.ipynb**] タブを横にドラッグするか、[**capstone_ja_jp.ipynb**] タブを選択 (右クリック) し、[**New View for Notebook**] をクリックします。これで Data Wrangler のフローを実行しながら、操作方法を表示できるようになりました。
- **オプション 2:** SageMaker Studio のタブを切り替えながら、手順を進めます。Data Wrangler の手順の調査を終了する場合は、[**capstone.ipynb**] タブをクリックしてノートブックに戻ります。

新しいフローファイルの作成時に **The following instance type is not available: ml.m5.4xlarge.Try selecting a different instance below.** (次のインスタンスタイプは利用できません: ml.m5.4xlarge。以下に示す別のインスタンスを選択してください) というエラーメッセージが表示される場合は、別のインスタンスタイプを選択します。代わりに **ml.m5.8xlarge** を試してみてください。

**An error occurred loading this view** (このビューを読み込む際にエラーが発生しました) と表示される場合は、[**untitled.flow**] タブを閉じ、ファイルブラウザからフローファイルを開き直してください。

**ヒント 1**: データセットを調査する方法はいくつもあります。SageMaker Data Wrangler のフローを開き、調査を開始できます。**claims.csv** と **customers.csv** の両方を、**databucket-** で始まる S3 バケットから Data Wrangler にインポートする必要があります。

**ヒント 2**: 2 つ目のテーブルをインポートするには、[**Data flow**] に戻り、[**Import**] タブをクリックして、別のデータセットをインポートします。

**ヒント 3**: Data Wrangler では、[**Get data insights**] と [**Add analysis**] の 2 つの方法でデータを調査できます。データを使ってサンプルグラフをいくつか表示した後、必要に応じてノートブックの他のプロットツールを使ってデータを分析をすることもできます。**plt** と **sns** のライブラリはインストールされています。使い慣れた分析ツールを自由に使って、データセットを調査してください。 

結合したデータセットから、より意味のある結果が得られましたか。

SageMaker Studio でデータセットを調査する方法については、「<a href="#task1-2-1" target="_self">**データセットを SageMaker Studio で確認する**</a>」を参照してください。

ノートブックでデータセットを調査する方法については、「<a href="#task1-2-2" target="_self">**データセットをノートブックで確認する**</a>」を参照してください。

データセットを調査し、実行する処理手順を確認したら、このタスクは完了です。

In [ ]:
# ここにタスク 1.2 のコードを追加する



### タスク 1.3: SageMaker Clarify のプロセッサジョブを使用してバイアスレポートを実行する

<a id="task1-3-continue"></a>

SageMaker Clarify を使用して、トレーニング前のバイアスレポートを実行し、データ内のクラスの不均衡を特定します。SageMaker Data Wrangler フローを使用して、SageMaker Studio でバイアスレポートを実行します。
1. まず、2 つのテーブルを結合します。

- 統合: **Inner** 結合で **claims.csv** を **customers.csv** に**policy_id**を基に統合してください。

**ヒント 1**: トレーニング前のバイアスレポートを作成するには、Data Wrangler フローに新しい分析を追加し、[**Analysis type**] で [**Bias Report**] を選択します。

**ヒント 2**: バイアスレポートを何度か実行し、その都度異なる特徴量を使って分析できます。

Data Wranglerを使用してテーブルを結合する方法の詳細な手順については、**付録** セクションの <a href="#task1-3-1" target="_self">**SageMaker Studio でのテーブルの結合 (タスク 1.3)**</a> を参照してください。

トレーニング前のバイアスレポートの実行方法については、<a href="#task1-3-2" target="_self">**トレーニング前のバイアスレポートを実行する (タスク 1.3)**</a> を参照してください。

トレーニング前のバイアスレポートを実行し、レポートを確認したら、このタスクは完了です。

### タスク 1.4: データを準備する

<a id="task1-4-continue"></a>

SageMaker Studio の SageMaker Data Wrangler を使用して、データセットを準備します。以下の変換を列挙していますが、他の変換も自由に使用してください。

- Join: **claims.csv** から **customers.csv** に **policy_id** に基づいて **Inner** 結合する。
- Encode categorical (one-hot エンコーディング): **authorities_contacted**、**collision_type**、**customer_gender**、**driver_relationship**、**incident_type**、**policy_state** をエンコードする。
- Ordinal encode: **customer_education**、**policy_liability**、**incident_severity**、**policy_report_available** をエンコードする。
- Parse column as type: **vehicle_claim** と **total_claim_amount** を **Float** から **Long** に変換する。
- Drop column: **customer_zip** を削除する。
- Move column: **fraud** を移動する (**Move to start** を使用)。
- Rename column: **collision_type_N/A** から **/** を削除と **driver_relationship_N/A**から **_** を削除する。

- Rename column:  **policy_id_0**を**policy_id**に変更.

**ヒント 1**: Data Wrangler の Join を使用して、保険金請求テーブルと顧客テーブルを結合します。

**ヒント 2**: **policy_id** 列に基づいてテーブルを結合します。

**ヒント 3**: [**Add transform**] オプションを使用して、変換を追加します。

モデルトレーニングに最も影響を与えるのはどの変換だと思いますか。

Data Wrangler を使用してデータを準備する方法については、「<a href="#task1-4-1" target="_self">**SageMaker Data Wrangler を使用してデータを準備する**</a>」を参照してください。

既に処理済みのデータのサンプルセットをインポートしたい場合には、「<a href="#task1-4-2" target="_self">**処理済みのデータのサンプルセットをインポートする**</a>」を参照してください。


## 課題 2: SageMaker Feature Store で特徴量グループを作成する

データセットの処理が完了したので、今後の分析に再利用できるように特徴量と特徴量グループを作成します。SageMaker Feature Store を使用して、作成した特徴量を特徴量グループに保存し、モデルをトレーニングする際にこれらの特徴量をクエリします。

このタスクを完了するために、以下のサブタスクを実行します。

1. SageMaker Feature Store に特徴量をエクスポートする。
2. Amazon Athena を使用してオフラインストアの特徴量グループをクエリする。

この課題の所要時間は約 *30* 分です。

### タスク 2.1: SageMaker Feature Store に特徴量をエクスポートする

<a id="task2-1-continue"></a>

SageMaker Data Wrangler の [**Export to**] 機能を使用して、カスタムの Jupyter Notebook を作成します。このノートブックでは、特徴量の定義と特徴量グループを作成し、レコードを特徴量グループに取り込みます。ノートブックで、以下の手順を実行します。

- record 列と event_time 列に値を設定する。
- ノートブックのセルを実行して、特徴量グループを作成する。
- ノートブックのセルを実行して、作成した特徴量グループを確認する。
- ノートブックのセルを実行して、特徴量グループにレコードを取り込む。

**ヒント 1**: これらの手順はすべて SageMaker Studio で行うことができます。特徴量グループの作成が完了したら、このノートブックに戻り、タスク 2.2 に進むことができます。

**ヒント 2**: カスタム変換を追加して、**event_time** 列を作成します。

**ヒント 3**: カスタム変換を追加するためのコードは以下のとおりです。
    


In [ ]:
import time
import datetime
import pandas as pd
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


**ヒント 4**: Data Wrangler フローの最後で [**+**] アイコンをクリックして [**Export to**] オプションをクリックし、[**SageMaker Feature Store (via JupyterNotebook)**] を選択します。

**ヒント 5**: **enable_online_store** の値を **True** から **False** に変更することで、オンラインストアを無効にできます。

SageMaker Feature Store を使用して、推論用ではなく、トレーニング用にレコードを保存し、クエリするにはどうすればよいでしょうか。

[**Export to**] オプションを使用して特徴量グループを作成する方法については、「<a href="#task2-1" target="_self">**Export to オプションを使用して特徴量グループを作成する**</a>」を参照してください。

特徴量グループを作成し、その特徴量グループにデータを取り込んだら、このタスクは完了です。

### タスク 2.2: Athena を使用してオフラインストアの特徴量グループをクエリする

<a id="task2-2-continue"></a>

Athena を使用してオフラインストアからレコードを抽出します。次の課題では、抽出したレコードをトレーニング用、テスト用、検証用のデータセットに分割します。

以下のコードセルを使用して Amazon Athena API 呼び出しを行います。Amazon Athena コンソールを使用してクエリを作成することもできますが、それはこのラボの範囲外です。

**ヒント 1**: **feature_group.athena_query()** で Athena クエリを作成し、**query.table_name** でテーブル名を取得できます。

**ヒント 2**: **query.run(query_string=query_string, output_location=output_location)** でクエリを実行し、**query.as_dataframe()** で戻り値を dataframe として読み取ることができます。

データセットのタイムライン上の異なるポイントで発生する特徴を追跡するには、**event_time** をどのように使用できますか。

Athena でオフラインストアからレコードを抽出する方法については、「<a href="#task2-2" target="_self">**Athena でオフラインストアからレコードを抽出する**</a>」を参照してください。

返された Athena クエリを dataframe 変数として保存したら、このタスクは完了です。

In [ ]:
#ここにタスク 2.2 のコードを追加する



## 課題 3: モデルをトレーニングする

これでモデルをトレーニングする準備が整いました。データをトレーニング用、テスト用、検証用のデータセットに分割し、モデルをトレーニングします。

先ほどこのデータで SageMaker Autopilot を実行したところ、**F1** が **0.616**、**正解率**が **0.978**、**AUC** が **0.918**、**再現率**が **0.539** となりました。SageMaker Autopilot が生成するメトリクスについての詳細は、「[Metrics] (https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html)」 を参照してください。

トレーニングやチューニングを行う際は、値が SageMaker Autopilot のスコアと同等か、それを上回るようにし、Amazon SageMaker Debugger がエラーを報告しないことを確認します。

このタスクを完了するために、以下のサブタスクを実行します。

- ExperimentとRunを作成する。
- データをトレーニング用、テスト用、検証用のデータセットに分割する。
- トレーニングジョブを設定して実行する。
    - 基本的なトレーニングジョブを開始する。
    - SageMaker Debugger を有効にしてトレーニングジョブを実行し、レポートを分析する (オプション)。
- ハイパーパラメータのチューニングを実行する。

この課題の所要時間は約 *110* 分です。


### タスク 3.1: ExperimentとRunを作成する

<a id="task3-1-continue"></a>

ExperimentとRunの両方に名前を付ける変数を設定します。
Experimentには **experiment_name**、**run_name** および **description** が必要です。

変数を作成する方法の詳細な手順については、**付録**セクションの <a href="#task3-1" target="_self">**実験と実行に名前を付ける (タスク 3.1)**</a> を参照してください。


In [ ]:
# ここにタスク 3.1 のコードを追加する



### タスク 3.2: データをトレーニング用、テスト用、検証用のデータセットに分割する

<a id="task3-2-continue"></a>

SageMaker Feature Store からクエリした特徴量を使用して、データをトレーニング用、テスト用、検証用のデータセットに分割します。

**ヒント 1**: **np.split** を使用して、データセットを 3 つのパーティションに分割します。

**ヒント 2**: **to_csv** を使用して、CSV ファイルを作成し、**S3Uploader.upload** を使用して Amazon S3 に作成したファイルを追加します。

**ヒント 3**: 分割の最終成果物は、**トレーニング**用と**検証**用の値を持つ **data_inputs** 変数となります。

データをトレーニング用、テスト用、検証用のデータセットに分割する方法については、「<a href="#task3-2" target="_self">**データをトレーニング用、テスト用、検証用のデータセットに分割する**</a>」を参照してください。

データをトレーニング用、テスト用、検証用のデータセットに分割したら、このタスクは完了です。

In [ ]:
#ここにタスク 3.2 のコードを追加する



### タスク 3.3: トレーニングジョブを設定して実行する

<a id="task3-3-continue"></a>

最初のトレーニングジョブを開始し、コンテナ、Estimator、ハイパーパラメータを設定します。そして、**fit()** でモデルをトレーニングします。より詳細なレポートを確認する場合は、**DebuggerHookConfig** で SageMaker Debugger を有効にしてください。

**ヒント 1**: バージョン **1.5-1** の **XGBoost** コンテナを使用します。

**ヒント 2**: 開始するには、**eta**、**gamma**、**max_depth**、**min_child_weight**、**num_round**、**objective**、**subsample** のハイパーパラメータを設定する必要があります。

**ヒント 3**: 課題 1 で作成した **data_inputs** をトレーニングジョブの **inputs** 値として使用します。

**ヒント 4**: inputs と experiment_config を設定することで、トレーニングジョブを構成できます。experiment_config には、**sagemaker_session**, **run_name**,**experiment_name**が含まれています。.

**ヒント 5**: SageMaker Debugger を使用する場合は、**DebuggerHookConfig**、**ProfilerConfig**、Debugger **rule** オブジェクトを設定する必要があります。

モデルのパフォーマンスと正解率に大きな影響を与えると思われるハイパーパラメータはどれですか。 どのハイパーパラメータを最初にチューニングする予定ですか。

基本的なトレーニングジョブを設定し、実行する方法については、「<a href="#task3-3-1" target="_self">**基本的なトレーニングジョブを設定し、実行する**</a>」を参照してください。

デバッガーを有効にしてトレーニングジョブを設定して実行し、レポートを分析する方法については、「<a href="#task3-3-2" target="_self">**SageMaker のデバッガーを有効にしてトレーニングジョブを設定して実行し、レポートを分析する**</a>」を参照してください。

1 つ以上のトレーニングジョブを実行したら、このタスクは完了です。

In [ ]:
# ここにタスク 3.3 のコードを追加する



### タスク 3.4: ハイパーパラメータのチューニング

<a id="task3-4-continue"></a>

トレーニングジョブを終了し、分析しました。その結果に基づいて、ハイパーパラメータの範囲をチューニングし、さらにトレーニングジョブを実行してモデルを改善します。

**ヒント 1**: 開始するには、**alpha**、**eta**、**max_depth**、**min_child_weight**、**num_round** のハイパーパラメータの範囲を設定します。

**ヒント 2**: **HyperparameterTuner** を実行する際には、分析結果に基づいて **objective_metric_name** と **objective_type** を必ず設定してください。

**ヒント 3**: SageMaker Autopilot の結果よりも改善されたかどうかを確認するには、**SageMaker リソース** の [**Experiments and Runs**] メニューを開きます。Runで、**メトリクス**を確認します。**ObjectiveMetric** は **F1** スコアの **0.616** よりも高く、**validation:auc** の**最終的な値**は SageMaker Autopilot スコアの **0.918** よりも高くなっているべきです。

ハイパーパラメータのチューニングを行ったとき、モデルのパフォーマンスを最も向上させたのはどれですか。

トレーニングハイパーパラメータの範囲を設定する方法については、「<a href="#task3-4" target="_self">**トレーニングハイパーパラメータの範囲を設定する**</a>」を参照してください。

トレーニングハイパーパラメータの範囲を設定し、さらにトレーニングジョブを開始したら、このタスクは完了です。

In [ ]:
# ここにタスク 3.4 のコードを追加する



## 課題 4: モデルのバイアスを評価する

モデルの学習が完了したので、Amazon SageMaker Clarify を使用してモデルを評価します。問題が見つかった場合は、検出された不均衡を解消して、モデルを再トレーニングします。

このタスクを完了するために、以下のサブタスクを実行します。

- トレーニングジョブからモデルを作成する。
- Clarify のモデルの設定を作成する。
- Clarify のバイアスの設定を作成する。
- Clarify のプロセッサジョブを使用して、バイアス、データ、モデルの各レポートを実行する。
- SageMaker Clarify で検出された不均衡を解消する (オプション)。
- モデルを再トレーニングする (オプション)。

この課題の所要時間は約 *80* 分です。

### タスク 4.1: トレーニングジョブからモデルを作成する

<a id="task4-1-continue"></a>

定義した **model_name**、**role**、**container_def** を使って **create_model** を呼び出し、XGBoost モデルを作成します。

**ヒント 1**: **xgb.create_model()** を呼び出し、モデルの名前を決めます。

**ヒント 2**: セッションを使用し、**model_name**、**role**、**container_def** を渡して **create_model** を呼び出します。

モデルを作成する方法については、「<a href="#task4-1" target="_self">**モデルを作成する**</a>」を参照してください。

モデルを作成したら、このタスクは完了です。

In [ ]:
# ここにタスク 4.1 のコードを追加する



### タスク 4.2: Clarify のモデルの設定を作成する

<a id="task4-2-continue"></a>

**SageMakerClarifyProcessor** を使用して、Clarify のモデルの設定を作成します。

**ヒント 1**: **instance_count** と **instance_type** を設定します。

**ヒント 2**: キャップストーンのラボの冒頭で作成した**ロール**と**セッション**を使用します。

Clarify のモデルの設定を作成する方法については、「<a href="#task4-2" target="_self">**Clarify のモデルの設定を作成する**</a>」を参照してください。

Clarify のモデルの設定を作成したら、このタスクは完了です。

In [ ]:
# ここにタスク 4.2 のコードを追加する



### タスク 4.3: Clarify のバイアスの設定を作成する

<a id="task4-3-continue"></a>

データの設定、モデルの設定、ラベルの設定、バイアスの設定を作成します。

**ヒント 1**: まず、**DataConfig** を作成し、入力パス、出力パス、ヘッダー、データセットタイプを設定します。

**ヒント 2**: 次に、**ModelConfig** を作成し、コンテンツと許可タイプ、モデル名、インスタンスタイプ、インスタンス数を選択します。

**ヒント 3**: 次に、**ModelPredictedLabelConfig** を作成し、確率のしきい値を設定します。

**ヒント 4**: 最後に **BiasConfig** を作成し、ラベルの値またはしきい値、ファセット名、ファセットの値またはしきい値を設定します。

バイアスレポートでは、どのファセットを最初に調査しますか。 バイアスの影響を特に受けやすい特徴はありますか。

Clarify のバイアスの設定を作成する方法については、「<a href="#task4-3" target="_self">**Clarify のバイアスの設定を作成する**</a>」を参照してください。

Clarify のバイアスの設定を作成したら、このタスクは完了です。

In [ ]:
# ここにタスク 4.3 のコードを追加する



### タスク 4.4: Clarify のプロセッサジョブを使用してバイアス、データ、モデルのレポートを実行する

<a id="task4-4-continue"></a>

バイアス、データ、モデルの各レポートについて、既にすべての設定を選択しました。では、レポートを実行してみましょう。

**ヒント 1**: **data_config**、**bias_config**、**model_predicted_label_config**、**model_config** を **run_bias** に渡します。

**ヒント 2**: **pre_training_methods** と **post_training_methods** も設定する必要があります。

SageMaker Clarify を使ってバイアス、データ、モデルのレポートを実行する方法については、「<a href="#task4-4" target="_self">**SageMaker Clarify を使ってバイアス、データ、モデルのレポートを実行する**</a>」を参照してください。

Clarify プロセッサジョブを使用してバイアス、データ、モデルの各レポートを実行したら、このタスクは完了です。

In [ ]:
# ここにタスク 4.4 のコードを追加する



### タスク 4.5: SageMaker Clarify で検出した不均衡を解消する (オプション)

<a id="task4-5-continue"></a>

SageMaker Clarify で検出された不均衡を解消する方法はいくつもあります。使い慣れた方法を使用してください。このラボでは、列の 1 つからバイアスを解消する SMOTE (Synthetic Minority Over-sampling Technique) の例を紹介します。

**ヒント 1**: 不均衡を解消し、再テストを行う場合は、新しくリサンプリングされたデータフレームを作成します。次のタスクでは、新しい CSV ファイルを作成し、アップロードします。

不均衡を解消する方法については、「<a href="#task4-5" target="_self">**不均衡を削除する**</a>」を参照してください。

SageMaker Clarify で検出された不均衡を解消したら、このタスクは完了です。

In [ ]:
# ここにタスク 4.5 のコードを追加する



### タスク 4.6: モデルを再トレーニングする (オプション)

<a id="task4-6-continue"></a>

新しいファイルを Amazon S3 にアップロードします。そして、新しいEstimatorを作成し、新しいデータで再トレーニングします。

**ヒント 1**: **s3_client.upload_file** を使用し、新しいファイルをバケットにアップロードします。

**ヒント 2**: **xgboost_starter_script.py** を使用し、**XGBoost** を呼び出します。そして、新しいデータで再トレーニングします。

再トレーニングしたモデルは、より高い F1 スコアになりましたか。 SageMaker Debugger を使用した場合、発見された問題をすべて解決できましたか。

モデルを再トレーニングする方法については、「<a href="#task4-6" target="_self">**モデルを再トレーニングする**</a>」を参照してください。

モデルを再トレーニングしたら、このタスクは完了です。

In [ ]:
# ここにタスク 4.6 のコードを追加する



## 課題 5: バッチ変換

モデルをデプロイする準備ができました。バッチレコードでバッチ変換ジョブを使用して、Amazon S3 に保存されている予測データと正解率データを確認します。そして、SageMaker インスタンスのいくつかをクリーンアップします。

このタスクを完了するために、以下のサブタスクを実行します。

- モデルのバッチ変換ジョブを作成する。
- Amazon S3 に保存されている予測データを確認する。
- SageMaker インスタンスをクリーンアップする (オプション)

この課題の所要時間は約 *40* 分です。

### タスク 5.1: モデルのバッチ変換ジョブを作成する

<a id="task5-1-continue"></a>

モデルEstimatorの**Transformer**を使用し、バッチ変換ジョブを作成します。そして、そのバッチジョブを実行します。

**ヒント 1**: Transformerを使用して、**instance_count**、**instance_type**、**strategy**、**assemble_with**、**output_path** を設定します。

**ヒント 2**: **test_path** に記載されているテストデータをエンドポイントに送信し、結果を待ちます。

バッチ変換ジョブを作成する方法については、「<a href="#task5-1" target="_self">**バッチ変換ジョブを作成する**</a>」を参照してください。

バッチ変換ジョブを作成し、レコードのセットで実行したら、このタスクは完了です。

In [ ]:
# ここにタスク 5.1 のコードを追加する



### タスク 5.2: Amazon S3 に保存されている予測データを確認する

<a id="task5-2-continue"></a>

バッチ変換ジョブが完了したら、Amazon S3 からデータを読み込みます。

**ヒント 1**: **%aws s3 cp --recursive $transformer.output_path ./** を使用して、Transformerの出力からデータをコピーできます。

**ヒント 2**: データを取得したら、**%head test_data_batch.csv.out** で確認できます。

予測値を見てみましょう。意外な予測値はありますか。

バッチ変換ジョブの予測データを確認する方法については、「<a href="#task5-2" target="_self">**バッチ変換ジョブの予測データと正解率データを確認する**</a>」を参照してください。

バッチ変換ジョブの予測データを確認したら、このタスクは完了です。

In [ ]:
# ここにタスク 5.2 のコードを追加する



### タスク 5.3: SageMaker インスタンスをクリーンアップする (オプション)

コストを低く抑えるためのベストプラクティスは、使わなくなったインスタンスを削除することです。SageMaker Studio を使用すると、インスタンスをすばやく削除できます。少し時間を取って SageMaker Studio で現在のリソースリストを開き、残っているインスタンスを削除してください。

次のパイプラインタスクを完了する予定がある場合は、**ノートブックインスタンスを実行したままにしておきます**。

**ヒント 1**: SageMaker Studio の **Running Terminals and Kernels** アイコンをクリックすると、実行中のインスタンスのリストを表示できます。

**ヒント 2**: **Shut down** アイコンをクリックするとインスタンスを停止できます。

<a id="task5-3-continue"></a>

SageMaker Studio で SageMaker インスタンスをクリーンアップする方法については、「<a href="#task5-3" target="_self">**SageMaker Studio で SageMaker インスタンスをクリーンアップする**</a>」を参照してください。

SageMaker Studio ですべての SageMaker インスタンスを停止したら、このタスクは完了です。

## 課題 6: 自動化されたパイプラインを構築する (オプション)

Amazon SageMaker SDK と Amazon SageMaker Studio を使用して機械学習 (ML) ワークフローを構築しました。次に、SageMaker Pipelines を使用してワークフローをスケールします。この課題では、ラボ環境のために用意されたパイプラインスクリプトを見ていきます。

- パイプラインステップを作成する。
    - SageMaker Feature Store から処理済みデータをクエリする。
    - モデルをトレーニングし、チューニングする。
    - トレーニングしたモデルを評価する。
    - バッチ変換ジョブを実行する。
    - モデルを登録する。
    - SageMaker Clarify でモデルトレーニングを評価する。
- パイプラインを定義し、開始する。
- 機械学習の系統追跡を表示する。

この課題の所要時間は約 *120* 分です。


### タスク 6.1: パイプラインを設定する

<a id="task6-1-continue"></a>

パイプラインのテンプレートを使用し、入力と出力を設定します。設定が完了したら、パイプラインを実行します。パイプラインには、さまざまなステップを含めることができます。以下は、設定に使用されるステップの推奨リストです。
- **AutoModelProcess**: .csv ファイルを取り込み、トレーニング用、テスト用、検証用のデータセットに分割する **Processing** ステップ。
- **AutoHyperParameterTuning**: ハイパーパラメータの範囲を指定し、モデルをチューニングする **Tuning** ステップ。
- **AutoEvalBestModel**: 最適なモデルを示す評価レポートを作成する **Processing** ステップ。
- **CheckAUCScoreAutoEvaluation**: 評価メトリクスに基づきモデルを評価する **Condition** ステップ。
- **AutoCreateModel**: モデルを作成する **Model** ステップ。
- **RegisterAutoModel-RegisterModel**: モデルを登録する **RegisterModel** ステップ。
- **AutoModelConfigFile**: バイアスレポートを作成する **Processing** ステップ。
- **AutoTransform**: バッチ変換ジョブを実行する **Transform** ステップ。
- **ClarifyProcessingStep**: SageMaker Clarify のジョブを実行する **Processing** ステップ。

**ヒント 1**: 多くのパイプラインステップから選択できます。パイプラインステップの詳細と各ステップのサンプルコードについては、「[Pipeline Steps](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)」を参照してください。

**ヒント 2**: まず、データを取り込むために **Processing** ステップから始めます。次に、**Tuning** ステップを作成し、モデルをチューニングします。そして、**Model** ステップを作成し、モデルを作成します。

**ヒント 3**: このチュートリアルには、評価レポートの作成、バイアスレポートの作成、バッチ変換ジョブの実行、SageMaker Clarify ジョブの実行の手順を含むサンプルソリューションが含まれています。

パイプラインを設定する方法については、「<a href="#task6-1" target="_self">**パイプラインを設定する**</a>」を参照してください。

パイプラインを設定し、パイプラインジョブを開始したら、このタスクは完了です。

In [ ]:
# ここにタスク 6.1 のコードを追加する


### タスク 6.2: パイプラインをモニタリングする

<a id="task6-2-continue"></a>

パイプラインの実行中に、入力と出力を表示し、パイプラインをモニタリングします。

**ヒント 1**: `RunPipeline.describe()` を使用して、作成したパイプラインを指定します。

**ヒント 2**: SageMaker Studio UI で実行中のパイプラインステップを表示できます。[**Home**] メニューを開いて [**Pipelines**] をクリックし、作成したパイプラインを選択します。

パイプラインをモニタリングする方法については、「<a href="#task6-2" target="_self">**パイプラインをモニタリングする**</a>」を参照してください。

パイプラインのモニタリングが終了したら、このタスクは完了です。

In [ ]:
# ここにタスク 6.2 のコードを追加する



お疲れ様でした。 自動車保険のデータセットを使用して、不正の可能性がある請求を検出しました。SageMaker Studio と SageMaker Python SDK を使用して、既存の自動車保険請求の中で不正の可能性があるものを予測するテクニカルソリューションを検討しました。

### クリーンアップ

このノートブックを完了しました。ラボの次の部分に進むために、以下を実行してください。

- このノートブックファイルを閉じる。
- ラボのセッションに戻り、「まとめ」を確認する。

## その他のリソース

- [Autopilot metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/autopilot-metrics-validation.html)
- [Processing step](https://docs.aws.amazon.com/sagemaker/latest/dg/build-and-manage-steps.html#step-type-processing)

## 付録

<a name="task1-1" id="task1-1"></a>

### 付録: データを表示する (タスク 1.1)

データを表示するには、Pandas を使用してパスを指定し、データを読み込みます。少し時間を取って両方のテーブルのサンプルを確認してください。

In [ ]:
# CSV ファイルを読み込む
claims_data = pd.read_csv("./data/claims_preprocessed.csv", index_col=0)
customers_data = pd.read_csv("./data/customers_preprocessed.csv", index_col=0)

In [ ]:
# 保険金請求データサンプル
claims_data.head()

In [ ]:
# 顧客データサンプル
customers_data.head()

<a href="#task1-1-continue" target="_self">タスク 1.1</a> に戻って、このラボを続けます。

<a name="task1-2-1" id="task1-2-1"></a>

### 付録: データセットを SageMaker Studio で確認する (タスク 1.2)

SageMaker Data Wrangler でデータ調査を開始します。S3 バケットからファイルをインポートし、データを分析します。

次のステップでは、SageMaker Studio の新しいタブが表示されます。以下のオプションの 1 つを実行して、手順を進めます。
- **オプション 1:** タブを並べて表示する。SageMaker Studio のメインウィンドウから分割画面ビューを作成するには、**capstone_ja_jp.ipynb** タブを横にドラッグするか、**capstone_ja_jp.ipynb** タブを選択し、ツールバーから **File** と **New View for Notebook** を選択します。これで、特徴量グループを探索する際に手順書を表示されながら実行することができます。
- **オプション 2:** SageMaker Studio のタブを切り替えながら、手順を進めます。

1. 左側にある [**SageMaker Home**] アイコンをクリックします。
1. [**Data**]タブをクリックし、 [**Data Wrangler**] をクリックします。

SageMaker Studio で [**Data Wrangler**] タブが表示されます。

1. **+** **Create Data Wrangler flow**を選択します。

SageMaker Studio で [**untitled.flow**] タブが表示されます。

1. **SageMaker Studio** で **untitled.flow** ファイルのタブが自動的に開くまで待ちます。これには **2～3** 分かかることがあります。

SageMaker Studio で **Create connection** ページが [**Data Wrangler**] タブに表示されます。

1. **untitled.flow** ファイルタブのコンテキストメニュー (右クリック) を開き、ファイル名を変更するには、**Rename Data Wrangler Flow...** を選択します。

SageMaker Studio で [**Rename File**] メッセージウィンドウが表示されます。

1. **New Name** で `CapstoneDataWrangler.flow` と入力します。
1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Rename</span> を選択します。

**Rename File** メッセージウィンドウが閉じられます。

1. [**CapstoneDataWrangler.flow**] タブの [**Available**] セクションで [**Amazon S3**] を選択します。

SageMaker Studio で [**CapstoneDataWrangler.flow**] タブで [**Import a dataset from S3**] ページが表示されます。

1. バケットリストで、**databucket** の名前が含まれているバケットを選択します。
1. ファイル名が **claims.csv** の 1 つ目のデータセットを選択します。

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span> を選択します。

1. **Data flow** ビューに戻り、[**CapstoneDataWrangler.flow**] タブの左上にある [**< Data flow**] を選択します。

1. [**CapstoneDataWrangler.flow**] タブの左上にある [**Import**] タブを選択します。

SageMaker Studio で [**Create connection**] ページが表示されます。

1. [**CapstoneDataWrangler.flow**] タブの [**Available**] セクションで [**Amazon S3**] を選択します。

SageMaker Studio で [**CapstoneDataWrangler.flow**] タブで [**Import a dataset from S3**] ページが表示されます。

1. バケットリストで、**databucket** の名前が含まれているバケットを選択します。
1. ファイル名が **customers.csv** の 2 つ目のデータセットを選択します。
1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Import</span> を選択します。

1. **Data flow** ビューに戻り、[**CapstoneDataWrangler.flow**] タブの左上にある [**< Data flow**] を選択します。

1. [Data flow] ビューで、上側の [**Data types**] アイコンの横にある [**+**] 記号をクリックして、[**Get data insights**] をクリックします。

1. レポートを作成し、インサイトを確認します。

1. [**Data flow**] ビューに戻り、先ほど選択していなかったファイルの[**Data types**] アイコンの横にある [**+**] 記号をクリックして、[**Add analysis**] をクリックします。

1. 分析を作成し、結果を確認します。

さまざまなレポートタイプを十分に活用してデータセットを調査します。調査が終わったら次のタスクに進むことができます。

<a href="#task1-2-continue" target="_self">タスク 1.2</a> に戻って、このラボを続けます。


<a name="task1-2-2" id="task1-2-2"></a>

### 付録: データセットをノートブックで確認する (タスク 1.2)

データセットを調査する方法はいくつもあります。ここでは、データ調査のステップの例をいくつか示します。これらを参考にして、さまざまな視点からデータセットを調査しましょう。

In [ ]:
# 性別グラフ
import matplotlib.pyplot as plt
customers_data.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

In [ ]:
# 不正被害グラフ
claims_data.fraud.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Not Fraud", "Fraud"]);

In [ ]:
# 教育カテゴリグラフ
educ = customers_data.customer_education.value_counts(normalize=True, sort=False)
plt.bar(educ.index, educ.values)
plt.xlabel("Customer Education Level");

In [ ]:
# 保険請求額グラフ
plt.hist(claims_data.total_claim_amount, bins=30)
plt.xlabel("Total Claim Amount")

In [ ]:
# 請求件数グラフ
customers_data.num_claims_past_year.hist(density=True)
plt.suptitle("Number of Claims in the Past Year")
plt.xlabel("Number of claims per year")

In [ ]:
# 支払いプロットグラフ
sns.pairplot(
    data=customers_data, vars=["num_insurers_past_5_years", "months_as_customer", "customer_age"]
);

In [ ]:
# 保険会社別不正被害グラフ
combined_data = customers_data.join(claims_data)
sns.lineplot(x="num_insurers_past_5_years", y="fraud", data=combined_data);

In [ ]:
# 月別の顧客グラフ
sns.boxplot(x=customers_data["months_as_customer"]);

In [ ]:
# 顧客の年齢グラフ
sns.boxplot(x=customers_data["customer_age"]);

In [ ]:
# 性別の不正被害グラフ
combined_data.groupby("customer_gender_female").mean()["fraud"].plot.bar()
plt.xticks([0, 1], ["Male", "Female"])
plt.suptitle("Fraud by Gender");

In [ ]:
# 相関行列グラフ
cols = [
    "fraud",
    "customer_gender_male",
    "customer_gender_female",
    "months_as_customer",
    "num_insurers_past_5_years",
]
corr = combined_data[cols].corr()

# 相関行列をプロットする
sns.heatmap(corr, annot=True, cmap="Reds");

In [ ]:
# 結合データを読み込む
combined_data = pd.read_csv("./data/claims_customer.csv")

In [ ]:
# 不要な列を削除する
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains("^Unnamed: 0")]
combined_data.head()

In [ ]:
# 結合データの統計情報の抽出
combined_data.describe()

In [ ]:
# 統計情報を生成する
combined_stats = []

for col in combined_data.columns:
    combined_stats.append(
        (
            col,
            combined_data[col].nunique(),
            combined_data[col].isnull().sum() * 100 / combined_data.shape[0],
            combined_data[col].value_counts(normalize=True, dropna=False).values[0] * 100,
            combined_data[col].dtype,
        )
    )

stats_df = pd.DataFrame(
    combined_stats,
    columns=["feature", "unique_values", "percent_missing", "percent_largest_category", "datatype"],
)
stats_df.sort_values("percent_largest_category", ascending=False)

In [ ]:
# ヒートマップグラフ
sns.set_style("white")

corr_list = [
    "customer_age",
    "months_as_customer",
    "total_claim_amount",
    "injury_claim",
    "vehicle_claim",
    "incident_severity",
    "fraud",
]

corr_df = combined_data[corr_list]
corr = round(corr_df.corr(), 2)

fix, ax = plt.subplots(figsize=(15, 15))

mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

ax = sns.heatmap(corr, mask=mask, ax=ax, annot=True, cmap="OrRd")

ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10, ha="right", rotation=45)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10, va="center", rotation=0)

plt.show()

<a href="#task1-2-continue" target="_self">タスク 1.2</a> に戻って、このラボを続けます。

<a name="task1-3-1" id="task1-3-1"></a>

### 付録: SageMaker Studioにおけるテーブルの結合 (Task 1.3)

1. **CapstoneDataWrangler.flow**タブの左上にある **< Data flow** を選択し、**Data flow**に戻る。
1. **claims.CSV Data types**アイコンの隣にある **+** サインを選び、コンテキストメニューから**Join**を選ぶ。

SageMaker Data Wranglerの**Join**が表示されます。

1. **customers.csv** の **Data types** アイコンを選択する

1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span>を選ぶ。

1. **Join Type**に**Inner**を選ぶ。

1. **Columns** セクション:
  - **Left**にて, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>を選択する.

  - **Right**にて, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>を選択する.

1.　<span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>を選択する。

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>を選択する.

<a href="#task1-3-continue" target="_self">タスク 1.3</a>に戻って、このラボを続けます。


<a name="task1-3-2" id="task1-3-2"></a>

### 付録: トレーニング前のバイアスレポートを実行する (タスク 1.3)

SageMaker Data Wrangler のフローを使用して、SageMaker Clarify のバイアスレポートを作成します。

1. **CapstoneDataWrangler.flow** タブを選択します。
1. **Data flow**ビューに移動します. もし必要であれば、**CapstoneDataWranglerLab.flow** タブの左上にある　**< Data flow** をクリックしてください。 
1. **Join**アイコンの横にある **+** サインを選びコンテキストメニューから**Add analysis**を選択する。

1. **Create analysis** セクションにて以下の設定をする:
- **Analysis type**にて, **Bias Report**を選択する。
- **Analysis name**にて、`fraud bias by age`と設定する。
- ターゲット列(Select the column your model predicsts(target))を選択します。このラボでは、ターゲット列は **fraud** です。
- **Is your predicted column a value or threshold?** にて, **value** オプションを選択する。
- 予測値(predicted value(s))を選択します。このラボでは、**fraud** の値に "**1**" を使用します。
- **Select the column to analyze for bias**にて, **customer_age**を選択します。

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Run</span>を選択します.

ジョブが終了したら、返されたメトリクスを表示します。バイアスがある場合はメモを取り、分析した列に対して適用する処理ステップを考えます。

バイアスを分析するすべての列に対して、この手順を繰り返します。

<a href="#task1-3-continue" target="_self">タスク 1.3</a> に戻って、このラボを続けます。

<a name="task1-4-1" id="task1-4-1"></a>

### 付録: SageMaker Data Wrangler を使用してデータを準備する (タスク 1.4)

データセットを結合するには、SageMaker Data Wrangler で **policy_id** を使用して結合します。

SageMaker Data Wranglerを使用して、フローの中の自由な箇所でデータを結合することができます。データを結合する前にそれぞれのデータをファイルごとに個別に準備することもできますし、結合してから特徴量を変換できます。SageMaker Data Wranglerのフローは柔軟に変更することができます。

もしtask 1.3にてテーブルを結合していなければ、以下のステップに従って処理を実行してください。

1. **Data flow**ビューに移動します。**CapstoneDataWranglerLab.flow** タブの左上にある　**< Data flow** をクリックしてください。

1. **claims.CSV Data types** アイコンの隣にある + サインを選び、コンテキストメニューから **Join** を選択する。

SageMaker Data Wranglerの **Join** ページが表示されます。

1. **customers.csv** の **Data types** アイコンを選択する。
1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Configure</span>を選択する。
1. **Join Type**にて, **Inner**を選択する。
1. **Columns** セクション:

    - **Left**にて, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>を選択する。
    
    - **Right**にて, <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">policy_id</span>を選択する。

1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>を選択する.

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>を選択する.

データのテーブルを結合することで、結合したデータを変換する。

1. **Data flow**ビューに移動します。**CapstoneDataWranglerLab.flow** タブの左上にある　**< Data flow** をクリックしてください。

1. **Join**　アイコンの横の **+**　サインを選択します。, コンテキストメニューから **Add transform**を選択します。

このメニューを使って、さらに変換を追加します。[Transformation] メニューの左側には、データセットのプレビューが表示されます。

SageMaker Data Wrangler フローに以下の変換ステップを加えます:
- Encode categorical (one-hot エンコーディング): **authorities_contacted**、**collision_type**、**customer_gender**、**driver_relationship**、**incident_type**、**policy_state** を、**Columns** という出力スタイルで、**Skip** 戦略を使用してエンコードする。
- Encode categorical (Ordinal encode): **Skip** 戦略を使用して、**customer_education**、**incident_severity**、**police_report_available**、**policy_liability** をエンコードする。
- Parse column as type: **vehicle_claim** と **total_claim_amount** を **Float** から **Long** に変換する。
- Manage columns(Drop column): **customer_zip** と **policy_id_1** を削除する。
- Manage columns(Move column): **fraud** を移動する (**Move to start** を使用)。
- Manage columns(Rename column): **collision_type_N/A** と **driver_relationship_N/A** の **/** 記号を **_** 記号に変更する。
- Manage columns(Rename column): **policy_id_0** を **policy_id** に変更する。

列名に **/** が含まれる場合は、**/** を **_** に置き換えて列名を変更する必要があります。例えば、one-hot エンコーディングで作成された列で、値が **N/A** となっている列は、列名を変更する必要があります。SageMaker Feature Store では、列名に **/** の文字が含まれる列は受け付けられません。

データの変換が完了し、モデルトレーニングを開始する準備ができたら、次のタスクに進むことができます。トレーニングやチューニングの間に問題を発見した場合は、いつでもこのフローに戻って変更できます。


<a href="#task1-4-continue" target="_self">タスク 1.4</a> に戻って、このラボを続けます。

<a name="task1-4-2" id="task1-4-2"></a>

### 付録: 処理済みのデータのサンプルセットをインポートする (タスク 1.4)

前処理がうまくいかない場合や、処理済みのデータセットを読み込む場合は、S3 バケットの data フォルダに保存されている処理済みデータにアクセスできます。

In [ ]:
#　処理済みデータのインポート
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims_customer.csv")
df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
df_processed.head()

<a href="#task1-4-continue" target="_self">タスク 1.4</a> に戻って、このラボを続けます。

<a name="task2-1" id="task2-1"></a>

### 付録: Export to オプションを使用して特徴量グループを作成する (タスク 2.1)

SageMaker Data Wrangler では、SageMaker Feature Store にデータをエクスポートできます。また、特徴量グループを設定し、その特徴量グループに変換されたデータを取り込むのに必要なすべてのコードを含むノートブックを作成できます。

1. [**Add step**] をクリックします。

1. [**Custom transform**] をクリックします。

1. [**Name**] に `event_time` と入力します。

1. **Python (PySpark)** を選択します (まだ選択されていない場合)。

1. **Your custom transform** で、以下のように入力します。


In [ ]:
import time
import datetime
from pyspark.sql.functions import lit
date_time = datetime.date.today()

df = df.withColumn("event_time", lit(time.mktime(date_time.timetuple())))


1. <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Preview</span>を選択します。

1. <span style="background-color:#57c4f8; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#00a0d2; border-radius:2px; margin-right:5px; white-space:nowrap">Add</span>を選択します。

これで、**event_time** がデータセットに列として追加されます。SageMaker Feature Store では、**event_time** と一意の **record** id が必要です。**record** id には、**policy_id** を使用します。

1. データフローに戻るには、[**< Data flow**] アイコンをクリックします。

1. SageMaker Data Wrangler で、一連の変換の横(event_time)にある [**+**] アイコンをクリックします。

1. [**Export to**] をクリックします。

1. [**SageMaker Feature Store (via Jupyter Notebook)**] を選択します。

新しいノートブックが開きます。

1. 最初のセルで、以下の変数を変更します:
- **record_identifier_feature_name** の **None** を "**policy_id**" に置き換えます。customer テーブルと claim テーブルを結合し、2 番目の policy_id 列を削除しなかった場合、**None** を "**policy_id_0**" に置き換える必要がある場合があります。**if** ステートメントにある **None** 値は変更しないでください。
- **event_time_feature_name** の **None** を "**event_time**" に置き換えます。**if** ステートメントにある **None** 値は変更しないでください。

セルの編集が完了すると、以下のようになります。:


In [ ]:
record_identifier_feature_name = "policy_id"
if record_identifier_feature_name is None:
   raise SystemExit("Select a column name as the feature group record identifier.")

event_time_feature_name = "event_time"
if event_time_feature_name is None:
   raise SystemExit("Select a column name as the event time feature name.")


2. すべてのセルを実行して、特徴量の定義と特徴量グループを作成し、処理ジョブを使用して変換されたデータを特徴量グループに取り込みます。 

セルの編集が完了したら、Feature Store を使用する準備が整います。

<a href="#task2-1-continue" target="_self">タスク 2.1</a> に戻って、このラボを続けます。

<a name="task2-2" id="task2-2"></a>

### 付録: Athena でオフラインストアからレコードを抽出する (タスク 2.2)

**athena_query** で Athena クエリを設定します。次に、**query_string** を設定します。そして、そのクエリを実行して結果のサンプルを表示します。

In [ ]:
# athena-query を設定し実行する
try:
    # 特徴量グループがある場合、その名前を取得する
    feature_group_name = sagemaker_session.boto_session.client("sagemaker", region_name=region).list_feature_groups()['FeatureGroupSummaries'][0]['FeatureGroupName']
    feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sagemaker_session)

    # Athena が設定済みであることを確認する
    try:
        boto3.client('athena').update_work_group(
            WorkGroup='primary',
            ConfigurationUpdates={
                'EnforceWorkGroupConfiguration':False
            }
        )
    except Exception:
        pass
    
    # クエリを設定する
    query = feature_group.athena_query()
    table = query.table_name
    query_string = f'SELECT * FROM "{table}" '
    output_location = f"s3://{sagemaker_session.default_bucket()}/query_results/"
    print(f"Athena query output location: \n{output_location}")

    # クエリを実行する
    query.run(query_string=query_string, output_location=output_location)
    query.wait()
    df_feature_store = query.as_dataframe()
    
    # 特徴量グループにデータが表示されるのを待つ
    attempts = 0
    while len(df_feature_store.index) == 0 and attempts < 30:
        print("Waiting for feature group to populate...")
        time.sleep(60)
        # クエリを再実行する
        query.run(query_string=query_string, output_location=output_location)
        query.wait()
        df_feature_store = query.as_dataframe()
        # 試行回数を増やす
        attempts += 1
    if len(df_feature_store.index) != 0:
        print("The feature group is populated.")
except IndexError as e:
    # 特徴量グループがない場合に、エラーを発生させる
    print("No feature groups were found. Please create a feature group.")

以上のコードブロックを実行した後に、Amazon Athenaのクエリレコードが**sagemaker**で名前のはじまるAmazon S3バケットに保存されます。保存されたクエリオブジェクトは**query_results**というディレクトリ内に保存されます。

<a href="#task2-2-continue" target="_self">タスク 2.2</a> に戻って、このラボを続けます。

<a name="task3-1" id="task3-1"></a>

### 付録: ExperimentとRunを作成する (タスク 3.1)

Experimentを作成するには、**sagemaker.experiments.run** を使用してexperimentを設定します。ここで、**experiment_name**、**run_name** と **description** を指定します。

In [ ]:
from time import gmtime, strftime

#experiment と run-namesの作成
create_date = strftime("%m%d%H%M")
capstone_experiment_name="capstone-experiment-{}".format(create_date)
capstone_run_name = "lab-capstone-run-{}".format(create_date)

# run_tagを定義
run_tags = [{'Key': 'lab-capstone', 'Value': 'lab-capstone-run'}]

# descriptionを追加
description="Using SM Experiments with the Auto dataset."

print(f"Experiment name - {capstone_experiment_name},  run name - {capstone_run_name}")

<a href="#task3-1-continue" target="_self">タスク 3.1</a> に戻って、このラボを続けます。

<a name="task3-2" id="task3-2"></a>

### 付録: データをトレーニング用、テスト用、検証用のデータセットに分割する (タスク 3.2)

データを分割するには、**np.split** を使用し、データの分割方法を指定します。そして、CSV ファイルを作成し、S3 バケットにアップロードします。次に、トレーニング用の入力を設定します。最後に、**data_inputs** 変数を作成します。この data_inputs は、課題を通じて、モデルトレーニング時にトレーニング用と検証用のデータセットを指定するために使用します。

In [ ]:
#トレーニング用、テスト用、検証用に分割する
try:
    # 特徴量グループがある場合、それを使用する
    df_feature_store = df_feature_store.iloc[: , :-4]
    df_processed_pre_split = df_feature_store
    print("Using the records from the feature group")
except NameError:
    # 特徴量グループがない場合、処理済みのデータセットを使用する
    df_processed = pd.read_csv("./data/claims_customer.csv", index_col=None)
    df_processed_pre_split = df_processed
    print("Using the processed records from Amazon S3")

# データをトレーニング用、検証用、テスト用のデータセットに分割する
train_data, validation_data, test_data = np.split(
    df_processed_pre_split.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_processed_pre_split)), int(0.9 * len(df_processed_pre_split))],
)

# CSV ファイルを作成し、デフォルトのバケットにアップロードする
train_data.to_csv("train_data.csv", index=False, header=False)
validation_data.to_csv("validation_data.csv", index=False, header=False)
test_data.to_csv("test_data.csv", index=False, header=False)

train_path = S3Uploader.upload("train_data.csv", "s3://{}/{}".format(bucket, prefix))
validation_path = S3Uploader.upload("validation_data.csv", "s3://{}/{}".format(bucket, prefix))
test_path = S3Uploader.upload("test_data.csv", "s3://{}/{}".format(bucket, prefix))

# トレーニング用の入力を設定する
train_input = TrainingInput(train_path, content_type="text/csv")
validation_input = TrainingInput(validation_path, content_type="text/csv")
test_input = TrainingInput(test_path, content_type="text/csv")

data_inputs = {
    "train": train_input,
    "validation": validation_input
}

<a href="#task3-2-continue" target="_self">タスク 3.2</a> に戻って、このラボを続けます。

<a name="task3-3-1" id="task3-3-1"></a>

### 付録: 基本的なトレーニングジョブを設定し、実行する (タスク 3.3)

基本的なトレーニングジョブから始める場合、ベーシックな **XGBoost** コンテナを使用します。そして、使用するコンテナとロールに注意しながらestimatorを設定します。これらの設定が完了したら、ハイパーパラメータを選択します。以下のコードで提供されるデフォルト値を使用してもいいですし、データ準備中に気付いたことに応じて編集していただいても構いません。

トレーニングジョブを実行するには、**fit()** を呼び出し、入力に **data_inputs** 変数を使用し、設定に**run_name**と**experiment_name** を使用します。

In [ ]:

from sagemaker import image_uris
# モデルをトレーニングする
# コンテナイメージを取得する
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# ハイパーパラメータを設定する
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8

hyperparameters = {
    "eta":eta,
    "gamma":gamma,
    "max_depth":max_depth,
    "min_child_weight":min_child_weight,
    "num_round":num_round,
    "objective":objective,
    "subsample":subsample
}

# Estimatorを設定する
xgb = sagemaker.estimator.Estimator(
    container,
    role,    
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags
)

with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)

# モデルをトレーニングする
    xgb.fit(
        inputs = data_inputs
    )        

<a href="#task3-3-continue" target="_self">タスク 3.3</a> に戻って、このラボを続けます。

<a name="task3-3-2" id="task3-3-2"></a>

### 付録: SageMaker のデバッガーを有効にしてトレーニングジョブを設定して実行し、レポートを分析する (タスク 3.3)

SageMaker Debugger を使用すると、ハイパーパラメータのチューニングに役立つ追加レポートがすばやく提供されるため、ハイパーパラメータの範囲を指定して、より多くトレーニングジョブを実行する際の時間を短縮できます。デバッガーを有効にするには、**DebuggerHookConfig** と **rules** を設定します。

In [ ]:
# デバッガーを有効にする
# コンテナイメージを取得する
container = sagemaker.image_uris.retrieve(
    region=boto3.Session().region_name, 
    framework="xgboost", 
    version="1.5-1"
)

# ハイパーパラメータを設定する
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=300
objective='binary:logistic'
subsample=0.7
        
hyperparameters = {
        "eta":eta,
        "gamma":gamma,
        "max_depth":max_depth,
        "min_child_weight":min_child_weight,
        "num_round":num_round,
        "objective":objective,
        "subsample":subsample
}

# Estimatorを設定する
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    base_job_name=base_job_name,
    instance_count=1, 
    instance_type="ml.m5.4xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    max_run=1800,
    hyperparameters=hyperparameters,
    tags = run_tags,

    # デバッガーのフック構成を設定する
    debugger_hook_config=DebuggerHookConfig(
        s3_output_path=bucket_path,  # Required
        collection_configs=[
            CollectionConfig(name="metrics", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="feature_importance", parameters={"save_interval": str(save_interval)},),
            CollectionConfig(name="full_shap", parameters={"save_interval": str(save_interval)}),
            CollectionConfig(name="average_shap", parameters={"save_interval": str(save_interval)}),
        ],
        ),
        # デバッガーのプロファイラー構成を設定する
        profiler_config = ProfilerConfig(
            system_monitor_interval_millis=500,
            framework_profile_params=FrameworkProfile()
    ),
        # デバッガーのルールオブジェクトを設定する
        rules = [
            ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
            Rule.sagemaker(rule_configs.create_xgboost_report()),  
            Rule.sagemaker(rule_configs.overfit()),
            Rule.sagemaker(rule_configs.overtraining()),
            Rule.sagemaker(rule_configs.loss_not_decreasing(),
                rule_parameters={
                    "collection_names": "metrics",
                    "num_steps": str(save_interval * 2),
                }
            )
    ]
)
with Run(
    experiment_name=capstone_experiment_name,
    run_name=capstone_run_name,
    sagemaker_session=sagemaker_session,
) as run:
    run.log_parameter("eta", eta)
    run.log_parameter("gamma", gamma)
    run.log_parameter("max_depth", max_depth)
    run.log_parameter("min_child_weight", min_child_weight)
    run.log_parameter("objective", objective)
    run.log_parameter("subsample", subsample)
    run.log_parameter("num_round", num_round)
# モデルをトレーニングする
xgb.fit(
    inputs = data_inputs
) 

<a href="#task3-3-continue" target="_self">タスク 3.3</a> に戻って、このラボを続けます。

<a name="task3-4" id="task3-4"></a>

### 付録: トレーニングハイパーパラメータの範囲を設定する (タスク 3.4)

少なくとも 1 つのモデルトレーニングが完了したので、データ処理と SageMaker Debugger から得た情報を使用して、ハイパーパラメータの範囲を選択できます。以下のハイパーパラメータ範囲を編集し、チューニングジョブを実行します。

In [ ]:
# モデルをチューニングする
# ハイパーパラメータの範囲を設定する
hyperparameter_ranges = {
    "alpha": ContinuousParameter(0, 2),
    "eta": ContinuousParameter(0, 1),
    "max_depth": IntegerParameter(1, 10),
    "min_child_weight": ContinuousParameter(1, 10),
    "num_round": IntegerParameter(100, 1000)
}

# ターゲットメトリクスとターゲットのタイプ (最大/最小) を定義する
objective_metric_name = "validation:auc"
objective_type="Maximize"

# HyperparameterTuner を定義する
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type="Auto"
)

# モデルをチューニングする
tuner.fit(
    inputs = data_inputs
)

<a href="#task3-4-continue" target="_self">タスク 3.4</a> に戻って、このラボを続けます。

<a name="task4-1" id="task4-1"></a>

### 付録: モデルを作成する (タスク 4.1)

XGBoost モデルを作成し、定義した **model_name**、**role**、**container_def** を指定して **create_model** を呼び出します。

In [ ]:
# モデルの設定
model_name = "capstone-clarify-model"
model = xgb.create_model(name=model_name)
container_def = model.prepare_container_def()
sagemaker_session.create_model(model_name, role, container_def)

<a href="#task4-1-continue" target="_self">タスク 4.1</a> に戻って、このラボを続けます。

<a name="task4-2" id="task4-2"></a>

### 付録: Clarify のモデルの設定を作成する (タスク 4.2)

**SageMakerClarifyProcessor** を使用して、Clarify のモデルの設定を作成します。**instance_count** と **instance_type** を設定します。キャップストーンのラボの冒頭で作成した **role** と **session** を使用します。

In [ ]:
# Clarify のプロセッサを定義する
clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge", 
    sagemaker_session=sagemaker_session
)

<a href="#task4-2-continue" target="_self">タスク 4.2</a> に戻って、このラボを続けます。

<a name="task4-3" id="task4-3"></a>

### 付録: Clarify のバイアスの設定を作成する (タスク 4.3)

Clarify のバイアス設定を作成するには、**label**、**headers**、**dataset_type** を設定するのに加え、データの出力パスを選択し、トレーニングジョブからの入力パスを設定します。

次に、**ModelConfig** と **ModelPredictedLabelConfig** を作成します。

最後に、SageMaker Clarify で確認するフィールドを含む **BiasConfig** を設定します。最初の結果に基づいて、確認するフィールドを追加または削除することができます。

In [ ]:
# データ設定を定義する
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=train_path,
    s3_output_path=bias_report_output_path,
    label="fraud",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)


In [ ]:
# モデル設定を定義する
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv",
)

In [ ]:
# ラベルの設定を定義する
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.8)

In [ ]:
# バイアスの設定を定義する
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="customer_gender_female", facet_values_or_threshold=[0], group_name="customer_age"
)

<a href="#task4-3-continue" target="_self">タスク 4.3</a> に戻って、このラボを続けます。

<a name="task4-4" id="task4-4"></a>

### 付録: SageMaker Clarify を使ってバイアス、データ、モデルのレポートを実行する (タスク 4.4)

Clarify のジョブが設定されたので、**run_bias** を呼び出してジョブを実行します。

In [ ]:
# バイアスレポートを実行する
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)

<a href="#task4-4-continue" target="_self">タスク 4.4</a> に戻って、このラボを続けます。

<a name="task4-5" id="task4-5"></a>

### 付録: 不均衡を解消する (タスク 4.5)

この例では、データセットのバイアスを抑制するために、**customer_gender_female** をアップサンプリングしています。他の特徴量にバイアスがある場合は、そうした特徴に対しても不均衡を解消することができます。**random_state** は `42` に設定されていますが、この値は変更することもできます。

In [ ]:
# 概要を表示する
gender = train_data["customer_gender_female"]
gender.value_counts()

In [ ]:
# 不均衡を削除する
sm = SMOTE(random_state=42)
train_data_upsampled, gender_res = sm.fit_resample(train_data, gender)
train_data_upsampled["customer_gender_female"].value_counts()

<a href="#task4-5-continue" target="_self">タスク 4.5</a> に戻って、このラボを続けます。

<a name="task4-6" id="task4-6"></a>

### 付録: モデルを再トレーニングする (タスク 4.6)

不均衡を検出し、新しいトレーニングデータセットを取得しました。このデータセットを使用して、ファイルを再トレーニングします。この操作を行うには、新しいファイルをアップロードし、新しいEstimatorを作成します。そして、**fit()** でそのデータを再トレーニングします。以下のコードには、サンプルとしていくつかのハイパーパラメータが含まれています。再トレーニングの際に、必要に応じてこれらを追加、削除、調整し、最適なモデルを見つけることができます。

In [ ]:
# アップサンプリングされた CSV ファイルをアップロードする
train_data_upsampled.to_csv("data/upsampled_train.csv", index=False, header=False)
retrain_path = S3Uploader.upload("data/upsampled_train.csv", "s3://{}/{}".format(bucket, prefix))
retrain_input = TrainingInput(retrain_path, content_type="text/csv")

retrain_data_inputs = {
    "train": retrain_input,
    "validation": validation_input
}

In [ ]:
# Estimatorを作成する
hyperparameters= {
    "max_depth": "5",
    "eta": "0.2",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "binary:logistic",
    "num_round": "300",
}

xgb_retrained = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type="ml.m5.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters
)

In [ ]:
# アップサンプリングされたデータを再トレーニングする
xgb_retrained.fit(
    inputs = retrain_data_inputs
) 

<a href="#task4-6-continue" target="_self">タスク 4.6</a> に戻って、このラボを続けます。

<a name="task5-1" id="task5-1"></a>

### 付録: バッチ変換ジョブを作成する (タスク 5.1)

モデル Estimatorを使用し、**Transformer** でバッチ変換ジョブを作成します。処理の効率を上げるため、strategy に **MultiRecord** を設定します。次に、**test_path** を渡し、推論が実行されるのを待ちます。

In [ ]:
# バッチTransformerを作成する 
# 再トレーニングされたモデルがある場合はそれを使用し、ない場合はオリジナルのモデルを使用する
try:
    model = xgb_retrained
except NameError:
    model = xgb

# Transformerを作成する
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m4.xlarge",
    strategy="MultiRecord",
    assemble_with="Line",
    accept="text/csv",
    output_path="s3://{}/{}/batch-transform/".format(bucket, prefix)
)

In [ ]:
# バッチ変換ジョブを実行する
test_data_batch = test_data.drop("fraud", axis=1)
test_data_batch.to_csv("test_data_batch.csv", index=False, header=False)
test_path_batch = S3Uploader.upload("test_data_batch.csv", "s3://{}/{}".format(bucket, prefix))

transformer.transform(test_path_batch, content_type="text/csv", split_type="Line", join_source="Input")
transformer.wait()

<a href="#task5-1-continue" target="_self">タスク 5.1</a> に戻って、このラボを続けます。

<a name="task5-2" id="task5-2"></a>

### 付録: バッチ変換ジョブの予測データと正解率データを確認する (タスク 5.2)

バッチ変換ジョブが終了したら、Amazon S3 に保存されている予測データを確認します。**Transformer**で設定した出力パスを参照し、データをサンプリングできます。

この出力では、各レコードの最後に**fraud**の予測値が付加されています。

In [ ]:
!aws s3 cp --recursive $transformer.output_path ./
test_data = pd.read_csv("test_data_batch.csv.out")
test_data.head()

<a href="#task5-2-continue" target="_self">タスク 5.2</a> に戻って、このラボを続けます。

<a name="task5-3" id="task5-3"></a>

### 付録: SageMaker Studio で SageMaker インスタンスをクリーンアップする (タスク 5.3)

SageMaker Studio でモデルを開発している間、クリーンアップすべきインスタンスがあるかどうか定期的に確認します。ある場合は、SageMaker Studio からインスタンスをシャットダウンできます。

1. 左のメニューバーから、[**Running Terminals and Kernels**] のアイコン (真ん中に四角がある円) をクリックします。

1. まだ開いているインスタンスがある場合は、各インスタンスタイプの右側にある [**Shut down**] アイコンをクリックします。

各インスタンスで実行中のアプリケーションを表示し、終了してもよいアプリケーションを確認できます。

1. ポップアップウィンドウが表示された場合は、[**Shut down all**] をクリックします。

1. リスト上にインスタンスが表示されなくなるまで、折を見て **Refresh List** アイコンをクリックします。インスタンスがシャットダウンされるまで 2～5 分かかることがあります。

**capstone_ja_jp.ipynb** ノートブックで使用しているインスタンスをシャットダウンする必要はありません。

<a href="#task5-3-continue" target="_self">タスク 5.3</a> に戻って、このラボを続けます。

<a name="task6-1" id="task6-1"></a>

### 付録: パイプラインを設定する (タスク 6.1)

パイプラインを作成するには、パイプラインプロセスの各ステップを定義してから、それを実行します。

この例では、以下のステップを作成します:
- **AutoModelProcess**: .csv ファイルを取り込み、トレーニング用、テスト用、検証用のデータセットに分割する **Processing** ステップ。
- **AutoHyperParameterTuning**: ハイパーパラメータの範囲を取得し、モデルをチューニングする **Tuning** ステップ。
- **AutoEvalBestModel**: 最適なモデルを示す評価レポートを作成する **Processing** ステップ。
- **CheckAUCScoreAutoEvaluation**: 評価メトリクスに基づきモデルを評価する **Condition** ステップ。
- **AutoCreateModel**: モデルを作成する **Model** ステップ。
- **RegisterAutoModel-RegisterModel**: モデルを登録する **RegisterModel** ステップ。
- **AutoModelConfigFile**: バイアスレポートを作成する **Processing** ステップ。
- **AutoTransform**: バッチ変換ジョブを実行する **Transform** ステップ。
- **ClarifyProcessingStep**: SageMaker Clarify のジョブを実行する **Processing** ステップ。

パイプラインの構築中にうまくいかなくなった場合は、以下のコードをカスタマイズするか、参照して独自のパイプラインを構築できます。

In [ ]:
# パイプラインを実行する
# 変数を設定する
model_name = "Auto-model"
sklearn_processor_version="0.23-1"
model_package_group_name="AutoModelPackageGroup"
pipeline_name= "AutoModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn',version=sklearn_processor_version,region=region)

# ファイルをデフォルトの S3 バケットにアップロードする
s3_client.put_object(Bucket=bucket,Key='data/')
s3_client.put_object(Bucket=bucket,Key='input/code/')
s3_client.upload_file(Filename="data/batch_data.csv", Bucket=bucket, Key="data/batch_data.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="data/claims_customer.csv", Bucket=bucket, Key="data/claims_customer.csv")  #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
s3_client.upload_file(Filename="pipelines/evaluate.py", Bucket=bucket, Key="input/code/evaluate.py")
s3_client.upload_file(Filename="pipelines/generate_config.py", Bucket=bucket, Key="input/code/generate_config.py")
s3_client.upload_file(Filename="pipelines/preprocess.py", Bucket=bucket, Key="input/code/preprocess.py")

# 重要な設定を行う。claims_customer.csv と batch_data.csv 以外の
# ファイルを使用する場合は、input_data を変更する
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
        name="ProcessingInstanceType",
        default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
        name="TrainingInstanceType",
        default_value="ml.m5.xlarge"
)
input_data = ParameterString(
        name="InputData",
        default_value="s3://{}/data/claims_customer.csv".format(bucket), 
)
batch_data = ParameterString(
        name="BatchData",
        default_value="s3://{}/data/batch_data.csv".format(bucket),
)

# scikit-learn スクリプトを実行して、SageMaker でデータ処理を行う
# SKLearnProcessor クラスを使用する
sklearn_processor = SKLearnProcessor(
        framework_version=sklearn_processor_version,
        instance_type=processing_instance_type.default_value, 
        instance_count=processing_instance_count,
        sagemaker_session=sagemaker_session,
        role=role,
)

# input_data を取り込むための処理ステップを設定する
step_process = ProcessingStep(
        name="AutoModelProcess",
        processor=sklearn_processor,
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train",\
                             destination=f"s3://{bucket}/output/train" ),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation",\
                            destination=f"s3://{bucket}/output/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test",\
                            destination=f"s3://{bucket}/output/test"),
            ProcessingOutput(output_name="batch", source="/opt/ml/processing/batch",\
                            destination=f"s3://{bucket}/data/batch"),
            ProcessingOutput(output_name="baseline", source="/opt/ml/processing/baseline",\
                            destination=f"s3://{bucket}/input/baseline")
        ],
        code=f"s3://{bucket}/input/code/preprocess.py",
        job_arguments=["--input-data", input_data],
)

# モデルのパス、コンテナイメージ uri、Estimatorのハイパーパラメータを設定する
model_path = f"s3://{bucket}/output"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.5-1",
    py_version="py3",
    instance_type=training_instance_type.default_value,
)

fixed_hyperparameters = {
    "eval_metric":"auc",
    "objective":"binary:logistic",
    "num_round":"100",
    "rate_drop":"0.3",
    "tweedie_variance_power":"1.4"
}

xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=model_path,
    base_job_name=f"auto-train",
    sagemaker_session=sagemaker_session,
    role=role,
)

# チューニングステップのハイパーパラメータ範囲を設定し、チューニングステップを設定する
hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "alpha": ContinuousParameter(0, 2),
    "max_depth": IntegerParameter(1, 10),
}
objective_metric_name = "validation:auc"

step_tuning = TuningStep(
    name = "AutoHyperParameterTuning",
    tuner = HyperparameterTuner(xgb_train, objective_metric_name, hyperparameter_ranges, max_jobs=2, max_parallel_jobs=2),
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },
)

# 評価のための処理ステップを設定する
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-auto-eval",
    role=role,
    sagemaker_session=sagemaker_session,
)

evaluation_report = PropertyFile(
    name="AutoEvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_eval = ProcessingStep(
    name="AutoEvalBestModel",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation",\
                            destination=f"s3://{bucket}/output/evaluation"),
    ],
    code=f"s3://{bucket}/input/code/evaluate.py",
    property_files=[evaluation_report],
)

# モデルの作成を設定する
model = Model(
    image_uri=image_uri,        
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    name=model_name,
    sagemaker_session=sagemaker_session,
    role=role,
)

inputs = CreateModelInput(
    instance_type="ml.m5.large",
    accelerator_type="ml.inf1.xlarge",
)

step_create_model = CreateModelStep(
    name="AutoCreateModel",
    model=model,
    inputs=inputs,
)

script_processor = ScriptProcessor(
    command=['python3'],
    image_uri=clarify_image,
    role=role,
    instance_count=1,
    instance_type=processing_instance_type,
    sagemaker_session=sagemaker_session,
)

bias_report_output_path = f"s3://{bucket}/clarify-output/bias"
clarify_instance_type = 'ml.m5.xlarge'
step_config_file = ProcessingStep(
    name="AutoModelConfigFile",
    processor=script_processor,
    code=f"s3://{bucket}/input/code/generate_config.py",
    job_arguments=["--modelname",step_create_model.properties.ModelName,"--bias-report-output-path",bias_report_output_path,"--clarify-instance-type",clarify_instance_type,\
                  "--default-bucket",bucket,"--num-baseline-samples","50","--instance-count","1"],
    depends_on= [step_create_model.name]
)

# バッチ変換ジョブを実行するためのステップを設定する
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    assemble_with="Line",
    accept="text/csv",    
    output_path=f"s3://{bucket}/AutoTransform"
)

step_transform = TransformStep(
    name="AutoTransform",
    transformer=transformer,
    inputs=TransformInput(data=batch_data,content_type="text/csv",join_source="Input",split_type="Line")
)

# SageMaker Clarify の処理ステップを設定する
analysis_config_path = f"s3://{bucket}/clarify-output/bias/analysis_config.json"

data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=f's3://{bucket}/output/train/train.csv', 
    s3_output_path=bias_report_output_path,
    label=0,
    headers=list(pd.read_csv("./data/claims_customer.csv", index_col=None).columns), #If you edit this, make sure to also edit the headers listed in generate_config to match your column names.
    dataset_type="text/csv",
)

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=role,
    instance_count=1,
    instance_type=clarify_instance_type,
    sagemaker_session=sagemaker_session,
)

config_input = ProcessingInput(
    input_name="analysis_config",
    source=analysis_config_path,
    destination="/opt/ml/processing/input/analysis_config",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_compression_type="None",
)

data_input = ProcessingInput(
    input_name="dataset",
    source=data_config.s3_data_input_path,
    destination="/opt/ml/processing/input/data",
    s3_data_type="S3Prefix",
    s3_input_mode="File",
    s3_data_distribution_type=data_config.s3_data_distribution_type,
    s3_compression_type=data_config.s3_compression_type,
)

result_output = ProcessingOutput(
    source="/opt/ml/processing/output",
    destination=data_config.s3_output_path,
    output_name="analysis_result",
    s3_upload_mode="EndOfJob",
)

step_clarify = ProcessingStep(
    name="ClarifyProcessingStep",
    processor=clarify_processor,
    inputs= [data_input, config_input],
    outputs=[result_output],
    depends_on = [step_config_file.name]
)

# モデル登録ステップを設定する
model_statistics = MetricsSource(
    s3_uri="s3://{}/output/evaluation/evaluation.json".format(bucket),
    content_type="application/json"
)
explainability = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
)

bias = MetricsSource(
    s3_uri="s3://{}/clarify-output/bias/analysis.json".format(bucket),
    content_type="application/json"
) 

model_metrics = ModelMetrics(
    model_statistics=model_statistics,
    explainability=explainability,
    bias=bias
)

step_register = RegisterModel(
    name="RegisterAutoModel",
    estimator=xgb_train,
    model_data=step_tuning.get_top_model_s3_uri(top_k=0,s3_bucket=bucket,prefix="output"),
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    model_metrics=model_metrics,
)

# モデル評価ステップを作成する
cond_lte = ConditionGreaterThan(
    left=JsonGet(
        step=step_eval,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.auc.value"
    ),
    right=0.75,
)

step_cond = ConditionStep(
    name="CheckAUCScoreAutoEvaluation",
    conditions=[cond_lte],
    if_steps=[step_create_model,step_config_file,step_transform,step_clarify,step_register],
    else_steps=[],
)

# パイプラインを定義する
def get_pipeline(
    region,
    role=None,
    default_bucket=None,
    model_package_group_name="AutoModelPackageGroup",
    pipeline_name="AutoModelPipeline",
    base_prefix = None,
    custom_image_uri = None,
    sklearn_processor_version=None
    ):
    """自動データで動作する SageMaker ML Pipeline インスタンスを取得する。
    引数:
        region: パイプラインを作成して実行する AWS リージョン
        role: ステップおよびパイプラインを作成/実行するための IAM ロール
        default_bucket: アーティファクトの保存に使用するバケット
    戻り値:
        パイプラインのインスタンス
    """

    # パイプラインのインスタンス
    pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            processing_instance_type,
            processing_instance_count,
            training_instance_type,
            input_data,
            batch_data,
        ],
        steps=[step_process,step_tuning,step_eval,step_cond],
        sagemaker_session=sagemaker_session
    )
    return pipeline


# パイプラインを作成する
pipeline = get_pipeline(
    region = region,
    role=role,
    default_bucket=bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

pipeline.upsert(role_arn=role)

# パイプラインを実行する
RunPipeline = pipeline.start()

<a href="#task6-1-continue" target="_self">タスク 6.1</a> に戻って、このラボを続けます。

<a name="task6-2" id="task6-2"></a>

### 付録: パイプラインをモニタリングする (タスク 6.2)

パイプラインの作成と実行が完了したので、パイプラインをモニタリングします。パイプラインのステータスは SageMaker Studio で確認できます。

パイプラインを削除する場合は、**delete_pipeline** で削除します。

In [ ]:
# パイプラインを記述する
RunPipeline.describe()

In [ ]:
# パイプラインをモニタリングする
RunPipeline.wait(delay=30, max_attempts=100)

In [ ]:
# パイプラインステップを一覧表示する
RunPipeline.list_steps()

In [ ]:
#パイプラインを削除する
# response = sagemaker_session.boto_session.client("sagemaker", region_name=region).delete_pipeline(PipelineName='AutoModelSMPipeline')
print(response)

<a href="#task6-2-continue" target="_self">タスク 6.2</a> に戻って、このラボを続けます。